In [15]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from gensim.corpora import Dictionary

from gensim.models import doc2vec
from collections import namedtuple
from gensim.test.utils import get_tmpfile

import pandas as pd

from tqdm import tqdm

from konlpy.tag import Twitter
twitter = Twitter()

In [4]:
# 경택이랑 데이터가 다른 듯
test = pd.read_excel('./test2.xlsx', encoding = 'utf-8') 

In [8]:
test.head()

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
0,201301,카페림보 - 김한민展 :: Exhibit Theater,http://www.arthub.co.kr/sub01/board03_view.htm...,['김한민(Kim Hanmin)'],2013-01-02 ▶ 2013-02-27,2013-02-15 / 16,Open 11:00 ~ Close 23:30,테이크아웃드로잉(Takeout Drawing) 이태원 다른전시 보기,서울시 용산구 이태원동 637,02-790-2637,www.takeoutdrawing.com,[''],● 카페림보 - 김한민展\n 생의 의미와 목적 그리고 사회에 대한 소속...
1,201301,박명희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['박명희(Park Myunghee)'],2013-01-15 ▶ 2013-01-27,없음,Open 9:00 ~ Close 20:00,세종갤러리(SEJONG GALLERY) 다른전시 보기,서울시 중구 충무로2가 61-3 세종호텔,02-3705-9021,www.sejonggallery.co.kr,[''],● 박명희展\n ‘꿈과 자연’이라는 주제로 시각적인 형상과 상상의 나래...
2,201301,잘 어질러진 방 : my place - 유소라展 :: Quilt,http://www.arthub.co.kr/sub01/board03_view.htm...,['유소라(Yu Sora)'],2013-01-30 ▶ 2013-02-19,2013-01-30 PM 5:00,Open 10:00 ~ Close 18:00(일요일 휴관),갤러리41(Gallery41) 다른전시 보기,서울시 종로구 사간동 41,02-744-0341,NaN,[''],● 잘 어질러진 방 : my place\n - 소소한 욕심들로 채워...
3,201301,부드러운 교란-백남준을 말하다 :: Media Art,http://www.arthub.co.kr/sub01/board03_view.htm...,"['백남준', '저드 얄커트', '만프레드 레베', '샬럿 무어만']",2013-01-29 ▶ 2013-06-30,2013-02-07 PM 5:00,"Open 10:00 ~ Close 18:00(2주, 4주 월요일 휴관)",백남준아트센터(NJP ART CENTER) 다른전시 보기,경기도 용인시 기흥구 상갈동 85,031-201-8529,www.njpartcenter.kr,[''],● 부드러운 교란-백남준을 말하다\n 백남준아트센터의 2013년도 첫 ...
4,201301,space-scape :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"['김민정', '김병주', '김용관']",2013-01-05 ▶ 2013-01-28,NaN,Open 10:30 ~ Close 20:00(주말 20:30),롯데갤러리 일산점 다른전시 보기,경기도 고양시 일산동구 장항동 784 롯데백화점 일산점 B1,031-909-2688,NaN,[''],● 세 가지 공간 그리고 하나의 풍경\n 저희 롯데갤러리 일산점에서는 ...


In [9]:
print(len(test))
test.drop_duplicates(['링크'], inplace=True)
print(len(test))

181
181


In [12]:
test.reset_index(drop=True, inplace=True)
print(len(test))
test.tail(5)

181


,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
176,201301,김재형展 :: Sculpture,http://www.arthub.co.kr/sub01/board03_view.htm...,['김재형(Kim Jaehyeong)'],2013-01-16 ▶ 2013-01-22,없음,Open 10:00 ~ Close 19:00,갤러리 이즈(Gallery IS) 다른전시 보기,서울시 종로구 관훈동 100-5,02-736-6669,www.galleryis.com,[''],"● 대상과 심상(心像)\n 김재형은 망치, 가위, 전동공구, 이불, T..."
177,201301,강성욱展 :: Sculpture,http://www.arthub.co.kr/sub01/board03_view.htm...,['강성욱(Kang Sunguck)'],2013-01-03 ▶ 2013-01-22,없음,Open 10:00 ~ Close 19:00(일요일 휴관),빛갤러리(VIT Gallery) 다른전시 보기,서울시 종로구 소격동 76 인곡빌딩,02-720-2250,NaN,[''],"● 상상(像像, 想像)을 블렌딩(Blending) 하다\n 강성욱은 1..."
178,201301,"화가의 여인, 나부裸婦 : 한국근현대누드걸작展 1930~2000 :: Painting",http://www.arthub.co.kr/sub01/board03_view.htm...,"['강대운', '구자승', '권옥연', '권진규', '김경', '김경승', '김정숙...",2013-01-31 ▶ 2013-02-20,없음,Open 10:30 ~ Close 19:30(백화점 휴점일 휴관),롯데갤러리 본점 다른전시 보기,서울시 중구 소공동 1번지 롯데백화점 본점 12F,02-726-4428~9,NaN,[''],"전시작가 ▶ 강대운, 구자승, 권옥연, 권진규, 김경, 김경승, 김정숙, \n ..."
179,201301,아이러니-복제된 형상구조 속 전복의 미학 - 관용展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['관용(Guan Yong 管勇)'],2013-01-25 ▶ 2013-02-21,2013-01-25 PM 5:30,Open 10:00 ~ Close 19:00,"인터알리아(Interalia) 아트스페이스 A, B공간 다른전시 보기",서울시 강남구 삼성동 147-17 레베쌍트 빌딩,02-3479-0151,NaN,[''],● 관용_아이러니_복제된 형상구조 속 전복의 미학\n Guan Yo...
180,201301,Lost in Translation - Bang & Lee展 :: Video,http://www.arthub.co.kr/sub01/board03_view.htm...,['Bang & Lee'],2013-01-02 ▶ 2013-01-31,없음,Open 8:30 ~ Close 19:00,"COMO(SKT타워, 을지로 입구역 4번 출구, 대전 SKT 둔산 사옥) 다른전시 보기",서울시 종로구 서린동 99 SK 본사빌딩,02-2121-7225,www.nabi.or.kr,[''],"● 데이터의 생산, 그리고 소비\n <FAQ>와 <Lost in Tra..."


In [22]:
analyzeDoc = namedtuple('AnalyzedDocument', 'words tags')
type(analyzeDoc)

type

In [23]:
docs = []

for i in tqdm(range(len(test['텍스트']))):
    temp = str(test['텍스트'][i])
    words = twitter.morphs(temp)
    tags = [i]
    docs.append(analyzeDoc(words, tags))

docs

100%|██████████| 181/181 [00:08<00:00, 20.88it/s]


[AnalyzedDocument(words=['●', '카페', '림보', '-', '김한민', '展', '생', '의', '의미', '와', '목적', '그리고', '사회', '에', '대한', '소속', '감', '을', '철저히', '잃어버린', '6', '명', '‘', '림보', '족', '’', '들이', '황폐', '함', '의', '나라', '제', '82', '국', '에서', '벌이는', '게릴라', '식', '저항', '과정', '을', '다룬', '그래픽', '노블', '<', '카페', '림보', '>(', '2012', ')', '를', '원작', '으로', '한', '창작', '극', '이', '테이크아웃', '드로잉', '-', '이태원동', '에서', '최초', '상연', '된', '다', '.', '2012', '년', '1', '월', '과', '2', '월', ',', '테이크아웃', '드로잉', '-', '이태원동', '은', '‘', '카페', '림보', '’', '가', '된', '다', '.', '작가', '김한민', '은', '극단', '장이', '되', '고', ',', '무대', '미술가', '이수연', '은', '큐레이터', '이자', '공간', '연출자', '가', '되', '며', '카페', '공간', '은', '연극', '의', '무대', '로', '변신', '한', '다', '.', '극장', '의', '형식', '에', '구애', '받', '음', '없이', '자유', '스러운', '연극', '공간', '의', '창', '출', '과', '즉흥', '적', '성격', '을', '강조한', '‘', '환경', '연극', '’', '의', '정신', '을', '이어', '받아', ',', '‘', '연극', '카페', '림보', '’', '는', '무대', '와', '객석', ',', '관람', '방식', '에', '대한', '고정', '된', '틀', '을', '깨고', '자', '한', '다', '.'

In [24]:
max_epochs = 20
vec_size = 200
alpha = 0.025

test_kkma_model = doc2vec.Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

In [25]:
test_kkma_model.build_vocab(docs)

In [30]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    test_kkma_model.train(docs,
                total_examples=test_kkma_model.corpus_count,
                epochs=test_kkma_model.iter)
    # decrease the learning rate
    test_kkma_model.alpha -= 0.0002
    # fix the learning rate, no decay
    test_kkma_model.min_alpha = test_kkma_model.alpha

iteration 0


/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19


In [32]:
test_data = twitter.morphs(test['텍스트'][2])

In [104]:
twitter.morphs(test['텍스트'][2])

['●',
 '잘',
 '어',
 '질',
 '러',
 '진',
 '방',
 ':',
 'my',
 'place',
 '-',
 '소소한',
 '욕심',
 '들',
 '로',
 '채워',
 '진',
 '장소',
 '가',
 '존재',
 '를',
 '이야기하다',
 '慾心',
 '과',
 '欲心',
 '이',
 '있다',
 '.',
 '慾心',
 '은',
 '대체로',
 '부정',
 '적',
 '인',
 '의미',
 '인데',
 ',',
 '欲心',
 '은',
 '‘',
 '하고',
 '자하',
 '는',
 '의욕',
 '’',
 '이란',
 '다',
 '.',
 '나',
 '는',
 '두',
 '가지',
 '욕심',
 '모두',
 '를',
 '그린',
 '다',
 '.',
 '소소한',
 '욕심',
 '(',
 '慾心',
 '과',
 '欲心',
 ')',
 '에서',
 '비롯된',
 '물건',
 '과',
 '이야기',
 '들',
 '은',
 ',',
 '내',
 '가',
 '없',
 '이',
 '도',
 '나',
 '의',
 '존재',
 '를',
 '증명해',
 '줄',
 '나',
 '의',
 '방이',
 '나',
 '작업실',
 '과',
 '같은',
 '공간',
 '을',
 '채운',
 '다',
 '.',
 '사람',
 '들',
 '의',
 '개인',
 '적',
 '인',
 '장소',
 '는',
 '각자',
 '의',
 '욕심',
 '이',
 '만들어',
 '내는',
 '것',
 '이라고',
 '생각한',
 '다',
 '.',
 '욕심',
 '이',
 '많은',
 '사람',
 '일',
 '수',
 '록',
 '더욱',
 '더',
 '개성',
 '있는',
 '장소',
 '를',
 '꾸리',
 '고',
 '있지',
 '않은',
 '가',
 '.',
 '나',
 '는',
 '이러한',
 '욕심',
 '이',
 '만들어',
 '낸',
 '세계',
 '와',
 '존재',
 '감',
 '을',
 '다양한',
 '각도',
 '에서',
 '바라보며',
 '욕심',
 '을'

In [33]:
new_vector = test_kkma_model.infer_vector(test_data)
sims = test_kkma_model.docvecs.most_similar([new_vector])
sims

[(2, 0.87160325050354),
 (79, 0.581635594367981),
 (116, 0.577156126499176),
 (97, 0.5764567852020264),
 (43, 0.551910400390625),
 (100, 0.5362223982810974),
 (57, 0.5342210531234741),
 (131, 0.5259295701980591),
 (130, 0.5192910432815552),
 (117, 0.5131046772003174)]

---

In [35]:
data = pd.read_csv('./[네이버미술백과] 서양화750.csv')

In [36]:
data.head()

,Unnamed: 0,artist,title,art_type,tech,movement,image,text
0,0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."


In [63]:
import numpy as np

In [41]:
del data['Unnamed: 0']
data

,artist,title,art_type,tech,movement,image,text
0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."
5,에펠탑의 신랑신부,마르크 샤갈,유화,캔버스에 유채(Huile sur toile),NaN,https://dthumb-phinf.pstatic.net/?src=%22https...,말년의 마르크 샤갈이 거주한 라콜린 저택 응접실의 벽난로 위로 그림 한 점이 걸려있...
6,아를의 반 고흐의 방,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,반 고흐는 《반 고흐의 방》이라는 이름으로 세 작품을 그렸다. 가장 오래된 버전은 ...
7,모나리자,레오나르도 다 빈치,유화,패널에 유채(Huile sur bois),르네상스,https://dthumb-phinf.pstatic.net/?src=%22https...,"15세기경, 패널에 유채로, 크기는 77x53cm이며 프랑스 파리의 루브르 박물관(..."
8,유디트,구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"살로메(Salome), 세멜레(Semele), 메데아(Medea), 그리고 유디트(..."
9,행복한 눈물,로이 릭텐스타인,유화,캔버스에 마그나펜(Magna on canvas),NaN,https://dthumb-phinf.pstatic.net/?src=%22https...,앤디 워홀과 함께 미국의 팝아트를 대표하는 로이 릭텐스타인(Roy Lichtenst...


In [77]:
len(data)

750

In [83]:
data.dropna(subset = ['text'], inplace = True)

In [87]:
data.columns = ['title','artist','art_type','tech','movement','image','text']
data.head()

,title,artist,art_type,tech,movement,image,text
0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."


In [103]:
grouped = data.groupby('artist').text.count()

grouped.sort_values(ascending=False)[:10]


artist
빈센트 반 고흐         38
클로드 오스카 모네       24
구스타프 클림트         21
피에르 오귀스트 르누아르    17
르네 마그리트          14
앙리 마티스           12
로이 릭텐스타인         10
레오나르도 다 빈치       10
마르크 샤갈           10
폴 고갱             10
Name: text, dtype: int64